In [1]:
%pylab inline
!ls /tuples

Populating the interactive namespace from numpy and matplotlib
ls: cannot access /tuples: No such file or directory


In [117]:
import numexpr
import numpy
import root_numpy
import pandas as pd
from rep import utils
from sklearn.ensemble import GradientBoostingClassifier
from rep.report.metrics import RocAuc
from rep.metaml import GridOptimalSearchCV, FoldingScorer, RandomParameterOptimizer, SubgridParameterOptimizer
from rep.estimators import SklearnClassifier, TMVAClassifier, XGBoostRegressor
from sklearn.ensemble import BaggingRegressor
from rep.metaml import RegressionParameterOptimizer
from rep.metaml import RandomParameterOptimizer
from sklearn.gaussian_process import GaussianProcess
from rep.report.metrics import OptimalMetric, ams

In [118]:
# importing data
sigbranches = ['B_s0_TAU_ps', 'mu_MINIPS', 'B_s0_IP_OWNPV', 'B_s0_doca', 'sum_isolation', 'B_s0_PT',
               'B_s0_cosnk', 'B_s0_CDFiso', 'mu_MINPT', 'B_s0_IPCHI2_OWNPV', 'B_s0_FD_OWNPV', 'B_s0_BDTS']#, 'mu_DeltaEta', 'mu_AbsPhi', 'B_s0_TAUCHI2',
              #'B_s0_IPS_OWNPV', 'B_s0_ACOSDIRA_OWNPV', 'B_s0_ENDVERTEX_CHI2', 'B_s0_IPCHI2_OWNPV', 'B_s0_FD_OWNPV']
bkgbranchesSS = sigbranches#+['B_s0_SSweight']
bkgbranchesOS = sigbranches


signaltrain = root_numpy.root2array("/tuples/MC_emu_S21p1_2.root", 
                             treename='DecayTree', 
                             branches=sigbranches)
#                             selection="B_s0_BDTS>0.05")
signaltrain = pd.DataFrame.from_records(signaltrain)


backgroundtrain = root_numpy.root2array("/tuples/BEmu_SS_S21p1_reweighted_2.root", 
                             treename='DecayTree', 
                             branches=bkgbranchesSS)
#                             selection="B_s0_BDTS>0.05")
backgroundtrain = pd.DataFrame.from_records(backgroundtrain)



    
signaltest = root_numpy.root2array("/tuples/MC_emu_S21p1_1.root", 
                             treename='DecayTree', 
                             branches=sigbranches)
#                             selection="B_s0_BDTS>0.05")
signaltest = pd.DataFrame.from_records(signaltest)


backgroundtest0 = root_numpy.root2array("/tuples/BEmu_OS_S21p1.root", 
                             treename='DecayTree', 
                             branches=bkgbranchesOS)
#                             selection="B_s0_BDTS>0.05")
backgroundtest = pd.DataFrame.from_records(backgroundtest0)



In [119]:
train_labels=concatenate((np.ones(signaltrain.shape[0]),
                          np.zeros(backgroundtrain.shape[0])))
test_labels=concatenate((np.ones(signaltest.shape[0]),
                          np.zeros(backgroundtest.shape[0])))


train_data = pd.concat([signaltrain, backgroundtrain], axis=0)
test_data = pd.concat([signaltest, backgroundtest], axis=0)


In [120]:
"""
re-implement this function since I cannot import it from sklearn.base
"""
# Authors: Alexandre Gramfort <alexandre.gramfort@inria.fr>
#          Mathieu Blondel <mathieu@mblondel.org>
#          Olivier Grisel <olivier.grisel@ensta.org>
#          Arnaud Joly <a.joly@ulg.ac.be>
#          Jochen Wersdorfer <jochen@wersdoerfer.de>
#          Lars Buitinck
#          Joel Nothman <joel.nothman@gmail.com>
#          Noel Dawe <noel@dawe.me>
# License: BSD 3 clause

from __future__ import division

import numpy as np

from sklearn.utils import check_array, check_consistent_length
from sklearn.utils.multiclass import type_of_target


def average_binary_score(binary_metric, y_true, y_score, average,
                          sample_weight=None):
    """Average a binary metric for multilabel classification
    Parameters
    ----------
    y_true : array, shape = [n_samples] or [n_samples, n_classes]
        True binary labels in binary label indicators.
    y_score : array, shape = [n_samples] or [n_samples, n_classes]
        Target scores, can either be probability estimates of the positive
        class, confidence values, or binary decisions.
    average : string, [None, 'micro', 'macro' (default), 'samples', 'weighted']
        If ``None``, the scores for each class are returned. Otherwise,
        this determines the type of averaging performed on the data:
        ``'micro'``:
            Calculate metrics globally by considering each element of the label
            indicator matrix as a label.
        ``'macro'``:
            Calculate metrics for each label, and find their unweighted
            mean.  This does not take label imbalance into account.
        ``'weighted'``:
            Calculate metrics for each label, and find their average, weighted
            by support (the number of true instances for each label).
        ``'samples'``:
            Calculate metrics for each instance, and find their average.
    sample_weight : array-like of shape = [n_samples], optional
        Sample weights.
    binary_metric : callable, returns shape [n_classes]
        The binary metric function to use.
    Returns
    -------
    score : float or array of shape [n_classes]
        If not ``None``, average the score, else return the score for each
        classes.
    """
    average_options = (None, 'micro', 'macro', 'weighted', 'samples')
    if average not in average_options:
        raise ValueError('average has to be one of {0}'
                         ''.format(average_options))

    y_type = type_of_target(y_true)
    if y_type not in ("binary", "multilabel-indicator"):
        raise ValueError("{0} format is not supported".format(y_type))

    if y_type == "binary":
        return binary_metric(y_true, y_score, sample_weight=sample_weight)

    check_consistent_length(y_true, y_score, sample_weight)
    y_true = check_array(y_true)
    y_score = check_array(y_score)

    not_average_axis = 1
    score_weight = sample_weight
    average_weight = None

    if average == "micro":
        if score_weight is not None:
            score_weight = np.repeat(score_weight, y_true.shape[1])
        y_true = y_true.ravel()
        y_score = y_score.ravel()

    elif average == 'weighted':
        if score_weight is not None:
            average_weight = np.sum(np.multiply(
                y_true, np.reshape(score_weight, (-1, 1))), axis=0)
        else:
            average_weight = np.sum(y_true, axis=0)
        if average_weight.sum() == 0:
            return 0

    elif average == 'samples':
        # swap average_weight <-> score_weight
        average_weight = score_weight
        score_weight = None
        not_average_axis = 0

    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))

    if y_score.ndim == 1:
        y_score = y_score.reshape((-1, 1))

    n_classes = y_score.shape[not_average_axis]
    score = np.zeros((n_classes,))
    for c in range(n_classes):
        y_true_c = y_true.take([c], axis=not_average_axis).ravel()
        y_score_c = y_score.take([c], axis=not_average_axis).ravel()
        score[c] = binary_metric(y_true_c, y_score_c,
                                 sample_weight=score_weight)

    # Average the results
    if average is not None:
        return np.average(score, weights=average_weight)
    else:
        return score

In [121]:
from sklearn.metrics import roc_curve, auc

def roc_auc_score(y_true, y_score, average="macro", sample_weight=None):
    def _binary_roc_auc_score(y_true, y_score, sample_weight=None):
        if len(np.unique(y_true)) != 2:
            raise ValueError("Only one class present in y_true. ROC AUC score "
                             "is not defined in that case.")

        fpr, tpr, tresholds = roc_curve(y_true, y_score,
                                        sample_weight=sample_weight)
    
        fpr_highRej = numpy.array([]) #fpr is false positive rate
        tpr_highRej = numpy.array([]) #tpr is true positive rate
        for index, element in enumerate(fpr):  #selecting only stuff with rej > 0.9

            if ((1 - element) > 0.9):
                fpr_highRej = np.append(fpr_highRej, element)
                tpr_highRej = np.append(tpr_highRej, tpr[index])

        return auc(fpr_highRej, tpr_highRej, reorder=True)

    return average_binary_score(
        _binary_roc_auc_score, y_true, y_score, average,
        sample_weight=sample_weight)

In [122]:
#from sklearn.metrics import roc_auc_score
from sklearn.base import clone

class MyScorer(object):
    def __init__(self, test_data, test_labels):
        self.test_data = test_data
        self.test_labels = test_labels
        
    def __call__(self, base_estimator, params, X, y, sample_weight=None):
        cl = clone(base_estimator)
        cl.set_params(**params)
        cl.fit(X, y)
        pred = cl.predict_proba(self.test_data)
        # Returning with minus, because we maximize metric
        return roc_auc_score(self.test_labels, pred[:,1])
        

In [123]:
from scipy.stats import randint, uniform
now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
old_stdout = sys.stdout
sys.stdout = open("training_stdout_{0}.txt".format(now), 'w')

In [ ]:
%%time
# define grid parameters
grid_param = {}
grid_param['Ntrees'] = randint(2000, 15000)
grid_param['MaxDepth'] = randint(2,5)
grid_param['MinNodeSize'] = ['0.2%', '0.3%', '0.5%', '1%', '2%', '3%']
grid_param['NCuts'] = randint(10,40)
grid_param['AdaBoostBeta'] = uniform(0.1, 0.4)




# using Gaussian Processes 
generator = RandomParameterOptimizer(grid_param, n_evaluations=24)

estimator = TMVAClassifier(method='kBDT', features=sigbranches)
scorer = MyScorer(test_data, test_labels)

grid_finder = GridOptimalSearchCV(estimator, generator, scorer, parallel_profile='threads-8')
grid_finder.fit(train_data, train_labels)

In [ ]:
import sys
sys.stdout = open('scores_{0}.txt'.format(now), 'w')
grid_finder.params_generator.print_results()

In [ ]:
plot(grid_finder.generator.grid_scores_.values())

In [ ]:
estim = grid_finder.fit_best_estimator(train_data, train_labels, sample_weight=None)

In [ ]:
report = estim.test_on(test_data, test_labels)

In [ ]:
figure(figsize(10, 8))
report.roc(physics_notion=True).plot(xlim=(0, 1), ylim=(0, 1))
plt.savefig("best_roc.pdf")

In [65]:
sys.stdout = old_stdout